### Load das bibliotecas

In [1]:
import pandas as pd
import pickle

### Load do dataset

In [2]:
dados_sage = pd.read_csv('../baseZIKA/lista_dados_sage_ORIGINAL.csv', sep=';')

In [3]:
dados_sage.head()

,ibge,uf,cidade,n_semana,n_ano,total_acumulado,total_investigacao,total_confirmado,total_descartado,total_obito_notificado,total_obito_investigado,total_obito_confirmado,total_obito_descartado
0,120001,AC,Acrelândia,6,2016,0,0,0,0,0,0,0,0
1,120005,AC,Assis Brasil,6,2016,0,0,0,0,0,0,0,0
2,120010,AC,Brasiléia,6,2016,0,0,0,0,0,0,0,0
3,120010,AC,Brasiléia,37,2016,1,1,0,0,0,0,0,0
4,120010,AC,Brasiléia,38,2016,1,1,0,0,0,0,0,0


### Ajustes nos dados:
* Cria coluna auxiliar **LastDayWeek** para calcular a data do último dia da respectiva semana;
* Cria coluna **n_mes** baseada no mês da coluna auxiliar (LastDayWeek);
* Cria coluna **total_confirmado_abs** com total absoluto de casos confirmados dentro de cada semana, uma vez que o total de casos confirmados original é cumulativo desde o início do ano;
* Remover as colunas que não são mais necessárias após a criação das colunas **n_mes** e **total_confirmado_abs**:
  * LastDayWeek (utilizada para a criação da coluna **n_mes**);
  * n_semana (será substituída pela coluna **n_mes**);
* Remove demais colunas com totais, pois serão descartadas no modelo (usaremos a coluna criada: **total_confirmado_abs**):
  * total_acumulado
  * total_investigacao
  * total_confirmado
  * total_descartado
  * total_obito_notificado
  * total_obito_investigado
  * total_obito_confirmado
  * total_obito_descartado

In [4]:
dados_sage2 = dados_sage
# Coluna n_mes
dados_sage2['LastDayWeek'] = pd.to_datetime((dados_sage['n_ano']).astype(str) + (dados_sage['n_semana']-1).astype(str) + "6", format="%Y%U%w")
dados_sage2['n_mes'] = pd.DatetimeIndex(dados_sage2['LastDayWeek']).month

# Coluna total_confirmado_abs
dados_sage2 = (dados_sage.assign(total_confirmado_abs = dados_sage.sort_values(['n_semana', 'n_ano'])
                                                                  .groupby(['ibge'])
                                                                  .total_confirmado.diff(1)))

# O registro da primeira semana de cada municipio nao encontrara um registro anterior para aplicar a funcao diff,
# retornando NaN.
# Nesse caso, assume-se o total confirmado da primeira semana deste municipio
dados_sage2.loc[dados_sage2.total_confirmado_abs.isnull(), 'total_confirmado_abs'] = dados_sage2['total_confirmado']
dados_sage2.total_confirmado_abs = dados_sage2.total_confirmado_abs.astype(int)

# Remocao de colunas
dados_sage2.drop(['total_acumulado', 'total_investigacao', 'total_confirmado', 'total_descartado',
                  'total_obito_notificado', 'total_obito_investigado', 'total_obito_confirmado',
                  'total_obito_descartado', 'LastDayWeek', 'n_semana'], axis=1, inplace = True)

In [5]:
# Soma dos ultimos totais confirmados (acumulados) de cada municipio na base original
(dados_sage.groupby('ibge').total_confirmado.last()).sum()

2293

In [6]:
# Soma dos totais confirmados (absolutos) de todos os municipios na base ajustada.
# Deve coincidir com o total da base original (acima)
dados_sage2.total_confirmado_abs.sum()

2293

In [7]:
dados_sage2

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
0,120001,AC,Acrelândia,2016,2,0
1,120005,AC,Assis Brasil,2016,2,0
2,120010,AC,Brasiléia,2016,2,0
3,120010,AC,Brasiléia,2016,9,0
4,120010,AC,Brasiléia,2016,9,0
5,120010,AC,Brasiléia,2016,9,0
6,120010,AC,Brasiléia,2016,10,0
7,120010,AC,Brasiléia,2016,10,0
8,120010,AC,Brasiléia,2016,10,0
9,120010,AC,Brasiléia,2016,10,0


In [8]:
dados_sage2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67243 entries, 0 to 67242
Data columns (total 6 columns):
ibge                    67243 non-null int64
uf                      67243 non-null object
cidade                  67243 non-null object
n_ano                   67243 non-null int64
n_mes                   67243 non-null int64
total_confirmado_abs    67243 non-null int64
dtypes: int64(4), object(2)
memory usage: 3.1+ MB


### Validação dos dados

O Município de código IBGE 292740 (Salvador) possui 44 registros, e por isso, foi o escolhido para a validação do reagrupamento.

A Validação consiste em confirmar se os dados de todas as semanas de um mesmo mês foram devidamente agrupados, indicando que o ajuste foi feito corretamente.

In [9]:
# Lista as colunas apos a criacao das novas colunas
dados_sage2.columns

Index(['ibge', 'uf', 'cidade', 'n_ano', 'n_mes', 'total_confirmado_abs'], dtype='object')

In [10]:
# Filtra o municipio a ser usado na validacao e reorganiza as colunas para exibir as novas colunas ao lado
# da coluna de origem (n_semana)
dados_sage2.loc[(dados_sage2.ibge == 292740)]

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
10704,292740,BA,Salvador,2016,2,60
10705,292740,BA,Salvador,2016,2,3
10706,292740,BA,Salvador,2016,2,2
10707,292740,BA,Salvador,2016,2,10
10708,292740,BA,Salvador,2016,3,15
10709,292740,BA,Salvador,2016,3,-11
10710,292740,BA,Salvador,2016,3,6
10711,292740,BA,Salvador,2016,3,8
10712,292740,BA,Salvador,2016,4,7
10713,292740,BA,Salvador,2016,4,9


In [11]:
dados_sage2.loc[dados_sage2.ibge == 292740].groupby(['ibge', 'uf', 'cidade', 'n_ano', 'n_mes']).n_mes.count()

ibge    uf  cidade    n_ano  n_mes
292740  BA  Salvador  2016   2        4
                             3        4
                             4        5
                             5        4
                             6        4
                             7        5
                             8        4
                             9        4
                             10       5
                             11       4
                             12       1
Name: n_mes, dtype: int64

Como podemos ver acima, dos 44 registros do município:
* 4 pertencem ao mês de fevereiro;
* 4 pertencem ao mês de março;
* 5 pertencem ao mês de abril;
* 4 pertencem ao mês de maio;
* 4 pertencem ao mês de junho;
* 5 pertencem ao mês de julho;
* 4 pertencem ao mês de agosto;
* 4 pertencem ao mês de setembro;
* 5 pertencem ao mês de outubro;
* 4 pertencem ao mês de novembro;
* 1 pertence ao mês de dezembro;

Dessa forma, ao término do reagrupamento desse município, deveremos ter 11 registros com o valor absoluto dos totais de cada mês.

In [12]:
aggregation_functions = {'ibge': 'first',
                         'uf': 'first',
                         'cidade': 'first',
                         'n_ano': 'first',
                         'n_mes': 'first',
                         'total_confirmado_abs': 'sum'}
dados_sage_agg = dados_sage2.groupby(['ibge', 'n_mes', 'n_ano']).aggregate(aggregation_functions)
dados_sage_agg = dados_sage_agg.reset_index(drop=True)

In [13]:
dados_sage_agg.loc[dados_sage_agg.ibge == 292740]

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
12240,292740,BA,Salvador,2016,2,75
12241,292740,BA,Salvador,2016,3,18
12242,292740,BA,Salvador,2016,4,28
12243,292740,BA,Salvador,2016,5,7
12244,292740,BA,Salvador,2016,6,9
12245,292740,BA,Salvador,2016,7,8
12246,292740,BA,Salvador,2016,8,2
12247,292740,BA,Salvador,2016,9,6
12248,292740,BA,Salvador,2016,10,1
12249,292740,BA,Salvador,2016,11,4


Validação bem sucedida!

In [14]:
dados_sage_agg

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
0,110001,RO,Alta Floresta D'Oeste,2016,10,0
1,110001,RO,Alta Floresta D'Oeste,2016,11,0
2,110001,RO,Alta Floresta D'Oeste,2016,12,0
3,110002,RO,Ariquemes,2016,10,0
4,110002,RO,Ariquemes,2016,11,0
5,110002,RO,Ariquemes,2016,12,0
6,110011,RO,Jaru,2016,10,0
7,110011,RO,Jaru,2016,11,0
8,110011,RO,Jaru,2016,12,0
9,110012,RO,Ji-Paraná,2016,2,0


In [15]:
dados_sage_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17728 entries, 0 to 17727
Data columns (total 6 columns):
ibge                    17728 non-null int64
uf                      17728 non-null object
cidade                  17728 non-null object
n_ano                   17728 non-null int64
n_mes                   17728 non-null int64
total_confirmado_abs    17728 non-null int64
dtypes: int64(4), object(2)
memory usage: 831.1+ KB


### Salvando os dados

In [16]:
# csv
dados_sage_agg.to_csv('../baseZIKA/lista_dados_sage_MENSAL.csv', index=False, sep=';')

# pickle
pickle.dump(dados_sage_agg, open('../baseZIKA/lista_dados_sage_MENSAL.p', "wb"))

### Validando a carga dos dados salvos

In [17]:
# csv
df1 = pd.read_csv('../baseZIKA/lista_dados_sage_MENSAL.csv', sep=';')

In [18]:
df1

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
0,110001,RO,Alta Floresta D'Oeste,2016,10,0
1,110001,RO,Alta Floresta D'Oeste,2016,11,0
2,110001,RO,Alta Floresta D'Oeste,2016,12,0
3,110002,RO,Ariquemes,2016,10,0
4,110002,RO,Ariquemes,2016,11,0
5,110002,RO,Ariquemes,2016,12,0
6,110011,RO,Jaru,2016,10,0
7,110011,RO,Jaru,2016,11,0
8,110011,RO,Jaru,2016,12,0
9,110012,RO,Ji-Paraná,2016,2,0


In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17728 entries, 0 to 17727
Data columns (total 6 columns):
ibge                    17728 non-null int64
uf                      17728 non-null object
cidade                  17728 non-null object
n_ano                   17728 non-null int64
n_mes                   17728 non-null int64
total_confirmado_abs    17728 non-null int64
dtypes: int64(4), object(2)
memory usage: 831.1+ KB


In [20]:
# pickle
df2 = pickle.load(open('../baseZIKA/lista_dados_sage_MENSAL.p', "rb"))

In [21]:
df2

,ibge,uf,cidade,n_ano,n_mes,total_confirmado_abs
0,110001,RO,Alta Floresta D'Oeste,2016,10,0
1,110001,RO,Alta Floresta D'Oeste,2016,11,0
2,110001,RO,Alta Floresta D'Oeste,2016,12,0
3,110002,RO,Ariquemes,2016,10,0
4,110002,RO,Ariquemes,2016,11,0
5,110002,RO,Ariquemes,2016,12,0
6,110011,RO,Jaru,2016,10,0
7,110011,RO,Jaru,2016,11,0
8,110011,RO,Jaru,2016,12,0
9,110012,RO,Ji-Paraná,2016,2,0


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17728 entries, 0 to 17727
Data columns (total 6 columns):
ibge                    17728 non-null int64
uf                      17728 non-null object
cidade                  17728 non-null object
n_ano                   17728 non-null int64
n_mes                   17728 non-null int64
total_confirmado_abs    17728 non-null int64
dtypes: int64(4), object(2)
memory usage: 831.1+ KB
